# Flux Balance Analysis
The purpose of the notebook is to run flux balance analysis to find the set of genome scale fluxes that maximizes biomass production.<br><br>
The flux balance analysis results are then compared to reaction rates determined by 13C-metabolic flux analysis (13C-MFA). <br><br>
The results of this notebook will be compared to transcript constained genome scale model flux results via E-Flux2. <br><br>
This notebook looks at glucose, glycerol, acetate, and oleic acid as sole carbon sources. <br><br>
In all cases, parsimonious flux balance analysis (pFBA) was used to prevent minimize degenerate solutions.


### Load imports

In [1]:
import cobra
import pandas as pd
import sys

source_dir = '../src'
sys.path.append(source_dir)
from add_column_to_13c_flux_df import add_column_to_13c_flux_df
from flux_prediction_scatterplot import flux_prediction_scatterplot

### Load the genome scale model

In [2]:
model = cobra.io.json.load_json_model("../genome_scale_models/iYLI647_corr.json")
model

Name,model
Memory address,10d80b7c0
Number of metabolites,1121
Number of reactions,1348
Number of genes,646
Number of groups,0
Objective expression,1.0*biomass_C - 1.0*biomass_C_reverse_c1d5c
Compartments,"c, e, m, n, x, r, g, v"


### Glucose GSM FBA

In [3]:
# minimal medium with glycerol as sole carbon source
model.medium = {
  'EX_glc(e)': 100.0, # 100 mmol of glucose
  'EX_h2o(e)': 10000.0,
  'EX_h(e)': 10000.0,
  'EX_nh4(e)': 10000.0,
  'EX_o2(e)': 10000.0,
  'EX_pi(e)': 10000.0,
  'EX_so4(e)': 10000.0,
}
  
with model:
  # update the model to use the specified medium
  medium = model.medium
  [print(model.medium[m], m) for m in model.medium]

  # find the optimal solution
  glucose_fba_solution = cobra.flux_analysis.pfba(model)

  # make a list of dictionaries with the reaction id, name, flux, and absolute flux
  reactions = []
  for reaction_id, flux in glucose_fba_solution.fluxes.items():
    reactions.append({
      'reaction_id': reaction_id,
      'reaction_name': model.reactions.get_by_id(reaction_id).name,
      'full_reaction': model.reactions.get_by_id(reaction_id).reaction,
      'flux': flux,
      'absolute_flux': abs(flux), # use for sorting, then drop
    })

  # make a dataframe from the list of dictionaries
  glucose_gsm_fba_df = pd.DataFrame(reactions)

  # sort the dataframe by absolute flux
  glucose_gsm_fba_df = glucose_gsm_fba_df.sort_values(by=['absolute_flux'], ascending=False)

  # drop the absolute flux column
  glucose_gsm_fba_df = glucose_gsm_fba_df.drop(columns=['absolute_flux'])

  # save the dataframe to a csv file
  glucose_gsm_fba_df.to_csv('../results/fba_gsm_fluxes/glucose_gsm_fba.csv', index=False)

  display(glucose_gsm_fba_df.head(30))


100.0 EX_glc(e)
10000.0 EX_h2o(e)
10000.0 EX_h(e)
10000.0 EX_nh4(e)
10000.0 EX_o2(e)
10000.0 EX_pi(e)
10000.0 EX_so4(e)


,reaction_id,reaction_name,full_reaction,flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-561.830307
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],341.609652
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],335.476108
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,326.533474
959,H2Ot,H2O transport via diffusion,h2o[e] <=> h2o[c],-325.265198
421,EX_h2o(e),H2O exchange,h2o[e] <=>,325.265198
617,CYOR_u6m,ubiquinol 6 cytochrome c reductase,2.0 ficytc[m] + 1.5 h[m] + q6h2[m] --> 2.0 foc...,286.766531
639,NADH2_u6cm,NADH dehydrogenase cytosolicmitochondrial,h[c] + nadh[c] + q6[m] --> nad[c] + q6h2[m],172.102739
957,CO2t,CO2 transporter via diffusion,co2[e] <=> co2[c],-167.905908
391,EX_co2(e),CO2 exchange,co2[e] -->,167.905908


### Glucose Central Metabolism FBA

In [4]:
# load glucose 13C MFA data
glucose_13c_mfa_df = pd.read_csv('../data/13c_mfa/glucose_13c_mfa.csv')

# only keep the reactions that are mapped to the GSM
glucose_13c_mfa_df.dropna(subset = ["reaction_ids"], inplace=True)

# add the GSM flux predictions to the 13C-MFA dataframe
glucose_central_fba_df = add_column_to_13c_flux_df(glucose_13c_mfa_df, glucose_gsm_fba_df, 'pFBA Flux')

# save the central fluxes as .csv
glucose_central_fba_df.to_csv('../results/fba_central_fluxes/glucose_central_fba.csv', index=False)

glucose_central_fba_df

,pathway,compartment,13C_mfa_reaction,reaction_ids,13C_mfa_flux,13C_mfa_flux_95%_lower_bound,13C_mfa_flux_95%_upper_bound,Forward Flux,Backward Flux,Stdev,pFBA Flux
0,substrate_uptake,cytosol,Glucose(substrate) + ATP == G6P,GLCt1,100.0000,100.0000,100.0000,[],[],0.0000,100.000000
1,substrate_uptake,cytosol,Glycerol(substrate) == GLYC,reverse_GLYCt,0.0000,0.0000,1.2414,[],[],0.3167,0.000000
2,substrate_uptake,cytosol,Acetate(substrate) == AC,ACt2r,0.0678,0.0000,0.7662,[],[],0.1955,0.000000
3,substrate_uptake,cytosol,oleic_acid(substrate) == AC,OCDCEAt,0.0000,0.0000,0.0000,[],[],0.0000,0.000000
4,emp,cytosol,G6P == F6P,PGI,72.6860,58.4274,78.3064,72.686,0,5.0712,35.574485
5,emp,cytosol,F6P + ATP == FBP,PFK or reverse_FBP,80.4599,71.9969,81.0889,149.0655,68.6056,2.3194,38.629379
6,emp,cytosol,FBP == DHAP + GAP,FBA,80.4599,71.9969,81.0889,162.3111,81.8512,2.3194,38.629379
7,emp,cytosol,DHAP == GAP,TPI,-19.5394,-25.9334,-18.2523,7.6658,27.2052,1.9595,54.583426
8,emp,cytosol,GAP == G3P + ATP + NADH,GAPD,66.2012,54.6225,66.7364,380.1085,313.9073,3.0903,125.955289
9,emp,cytosol,G3P == PEP,ENO,65.6276,53.6544,66.0194,1469.4,1403.8,3.1543,116.254215


### Glycerol GSM FBA

In [5]:
# minimal medium with glycerol as sole carbon source
model.medium = {
  'EX_glyc(e)': 100.0, # 100 mmol of glycerol
  'EX_h2o(e)': 10000.0,
  'EX_h(e)': 10000.0,
  'EX_nh4(e)': 10000.0,
  'EX_o2(e)': 10000.0,
  'EX_pi(e)': 10000.0,
  'EX_so4(e)': 10000.0,
}
  
with model:
  # update the model to use the specified medium
  medium = model.medium
  [print(model.medium[m], m) for m in model.medium]

  # find the optimal solution
  glycerol_fba_solution = cobra.flux_analysis.pfba(model)

  # make a list of dictionaries with the reaction id, name, flux, and absolute flux
  reactions = []
  for reaction_id, flux in glycerol_fba_solution.fluxes.items():
    reactions.append({
      'reaction_id': reaction_id,
      'reaction_name': model.reactions.get_by_id(reaction_id).name,
      'full_reaction': model.reactions.get_by_id(reaction_id).reaction,
      'flux': flux,
      'absolute_flux': abs(flux), # use for sorting, then drop
    })

  # make a dataframe from the list of dictionaries
  glycerol_gsm_fba_df = pd.DataFrame(reactions)

  # sort the dataframe by absolute flux
  glycerol_gsm_fba_df = glycerol_gsm_fba_df.sort_values(by=['absolute_flux'], ascending=False)

  # drop the absolute flux column
  glycerol_gsm_fba_df = glycerol_gsm_fba_df.drop(columns=['absolute_flux'])

  # save the dataframe to a csv file
  glycerol_gsm_fba_df.to_csv('../results/fba_gsm_fluxes/glycerol_gsm_fba.csv', index=False)

  display(glycerol_gsm_fba_df.head(30))


100.0 EX_glyc(e)
10000.0 EX_h2o(e)
10000.0 EX_h(e)
10000.0 EX_nh4(e)
10000.0 EX_o2(e)
10000.0 EX_pi(e)
10000.0 EX_so4(e)


,reaction_id,reaction_name,full_reaction,flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-574.850423
421,EX_h2o(e),H2O exchange,h2o[e] <=>,354.104943
959,H2Ot,H2O transport via diffusion,h2o[e] <=> h2o[c],-354.104943
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],315.462030
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,313.155793
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],312.569434
617,CYOR_u6m,ubiquinol 6 cytochrome c reductase,2.0 ficytc[m] + 1.5 h[m] + q6h2[m] --> 2.0 foc...,258.825073
639,NADH2_u6cm,NADH dehydrogenase cytosolicmitochondrial,h[c] + nadh[c] + q6[m] --> nad[c] + q6h2[m],136.786713
964,O2t,o2 transport diffusion,o2[e] <=> o2[c],132.078744
444,EX_o2(e),O2 exchange,o2[e] <=>,-132.078744


### Glycerol Central Metabolism FBA

In [7]:
# load glycerol 13C MFA data
glycerol_13c_mfa_df = pd.read_csv('../data/13c_mfa/glycerol_13c_mfa.csv')

# only keep the reactions that are mapped to the GSM
glycerol_13c_mfa_df.dropna(subset = ["reaction_ids"], inplace=True)

# add the GSM flux predictions to the 13C-MFA dataframe
glycerol_central_fba_df = add_column_to_13c_flux_df(glycerol_13c_mfa_df, glycerol_gsm_fba_df, 'pFBA Flux')

# save the central fluxes as .csv
glycerol_central_fba_df.to_csv('../results/fba_central_fluxes/glycerol_central_fba.csv', index=False)

glycerol_central_fba_df

,pathway,compartment,13C_mfa_reaction,reaction_ids,13C_mfa_flux,13C_mfa_flux_95%_lower_bound,13C_mfa_flux_95%_upper_bound,Forward Flux,Backward Flux,Stdev,pFBA Flux
0,substrate_uptake,cytosol,Glucose(substrate) + ATP == G6P,GLCt1,0,NaN,NaN,[],[],NaN,0.000000
1,substrate_uptake,cytosol,Glycerol(substrate) == GLYC,reverse_GLYCt,100,NaN,NaN,[],[],NaN,100.000000
2,substrate_uptake,cytosol,Acetate(substrate) == AC,ACt2r,0,NaN,NaN,[],[],NaN,0.000000
3,substrate_uptake,cytosol,oleic_acid(substrate) == AC,OCDCEAt,0,NaN,NaN,[],[],NaN,0.000000
4,emp,cytosol,G6P == F6P,PGI,6,NaN,NaN,420.0208,426.1404,NaN,-27.963948
5,emp,cytosol,F6P + ATP == FBP,PFK or reverse_FBP,9,NaN,NaN,0,8.679,NaN,-18.670033
6,emp,cytosol,FBP == DHAP + GAP,FBA,9,NaN,NaN,14.8765,23.5555,NaN,-18.670033
7,emp,cytosol,DHAP == GAP,TPI,90,NaN,NaN,398.4248,307.9967,NaN,80.869854
8,emp,cytosol,GAP == G3P + ATP + NADH,GAPD,80,NaN,NaN,79.8354,0.01,NaN,69.140862
9,emp,cytosol,G3P == PEP,ENO,74,NaN,NaN,1.00E+06,999900,NaN,39.780985


### Oleic Acid GSM FBA

In [8]:
# minimal medium with oleic acid as sole carbon source
model.medium = {
  'EX_ocdcea(e)': 10.0, # 10 mmol of oleic acid (to prevent maxing out reactions with 1000 max flux)
  'EX_h2o(e)': 10000.0,
  'EX_h(e)': 10000.0,
  'EX_nh4(e)': 10000.0,
  'EX_o2(e)': 10000.0,
  'EX_pi(e)': 10000.0,
  'EX_so4(e)': 10000.0,
}

with model:
  # update the model to use the specified medium
  medium = model.medium
  [print(model.medium[m], m) for m in model.medium]

  # find the optimal solution
  acetate_fba_solution = cobra.flux_analysis.pfba(model)

  # make a list of dictionaries with the reaction id, name, flux, and absolute flux
  reactions = []
  for reaction_id, flux in acetate_fba_solution.fluxes.items():
    reactions.append({
      'reaction_id': reaction_id,
      'reaction_name': model.reactions.get_by_id(reaction_id).name,
      'full_reaction': model.reactions.get_by_id(reaction_id).reaction,
      'flux': flux * 10, # flux normalized to 100 mols of oleic acid
      'absolute_flux': abs(flux), # use for sorting, then drop
    })

  # make a dataframe from the list of dictionaries
  oleic_acid_gsm_fba_df = pd.DataFrame(reactions)

  # sort the dataframe by absolute flux
  oleic_acid_gsm_fba_df = oleic_acid_gsm_fba_df.sort_values(by=['absolute_flux'], ascending=False)

  # drop the absolute flux column
  oleic_acid_gsm_fba_df = oleic_acid_gsm_fba_df.drop(columns=['absolute_flux'])

  # save the dataframe to a csv file
  oleic_acid_gsm_fba_df.to_csv('../results/fba_gsm_fluxes/oleic_acid_gsm_fba.csv', index=False)

  display(oleic_acid_gsm_fba_df.head(30))


10000.0 EX_h2o(e)
10000.0 EX_h(e)
10000.0 EX_nh4(e)
10000.0 EX_o2(e)
10.0 EX_ocdcea(e)
10000.0 EX_pi(e)
10000.0 EX_so4(e)


,reaction_id,reaction_name,full_reaction,flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-3584.266821
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,2269.151708
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],2265.438412
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],2265.438412
617,CYOR_u6m,ubiquinol 6 cytochrome c reductase,2.0 ficytc[m] + 1.5 h[m] + q6h2[m] --> 2.0 foc...,2053.128358
798,MALOAAtp,malateoxaloacetate shuttle,mal_L[x] + oaa[c] <=> mal_L[c] + oaa[x],1383.231157
444,EX_o2(e),O2 exchange,o2[e] <=>,-1382.166538
964,O2t,o2 transport diffusion,o2[e] <=> o2[c],1382.166538
624,MDH,malate dehydrogenase,mal_L[c] + nad[c] <=> h[c] + nadh[c] + oaa[c],1289.403152
691,H2Otp,H2O transport peroxisomal,h2o[c] <=> h2o[x],1051.172760


### Oleic Acid Central Metabolism FBA

In [9]:
# load oleic acid 13C MFA data
oleic_acid_13c_mfa_df = pd.read_csv('../data/13c_mfa/oleic_acid_13c_mfa.csv')

# only keep the reactions that are mapped to the GSM
oleic_acid_13c_mfa_df.dropna(subset = ["reaction_ids"], inplace=True)

# add the GSM flux predictions to the 13C-MFA dataframe
oleic_acid_central_fba_df = add_column_to_13c_flux_df(oleic_acid_13c_mfa_df, oleic_acid_gsm_fba_df, 'pFBA Flux')

# save the central fluxes as .csv
oleic_acid_central_fba_df.to_csv('../results/fba_central_fluxes/oleic_acid_central_fba.csv', index=False)

oleic_acid_central_fba_df

,pathway,compartment,13C_mfa_reaction,reaction_ids,13C_mfa_flux,13C_mfa_flux_95%_lower_bound,13C_mfa_flux_95%_upper_bound,Forward Flux,Backward Flux,Stdev,pFBA Flux
0,substrate_uptake,cytosol,Glucose(substrate) + ATP == G6P,GLCt1,0.0,NaN,NaN,[],[],NaN,0.000000
1,substrate_uptake,cytosol,Glycerol(substrate) == GLYC,reverse_GLYCt,0.0,NaN,NaN,[],[],NaN,0.000000
2,substrate_uptake,cytosol,Acetate(substrate) == AC,ACt2r,0.0,NaN,NaN,[],[],NaN,0.000000
3,substrate_uptake,cytosol,oleic_acid(substrate) == AC,OCDCEAt,100.0,NaN,NaN,[],[],NaN,100.000000
4,emp,cytosol,G6P == F6P,PGI,-77.4,NaN,NaN,70.1415,147.3525,NaN,-30.701311
5,emp,cytosol,F6P + ATP == FBP,PFK or reverse_FBP,-65.7,NaN,NaN,7.80E+04,7.81E+04,NaN,-69.437259
6,emp,cytosol,FBP == DHAP + GAP,FBA,-65.7,NaN,NaN,252.2214,318.2931,NaN,-69.437259
7,emp,cytosol,DHAP == GAP,TPI,-65.7,NaN,NaN,0,66.0717,NaN,-72.351056
8,emp,cytosol,GAP == G3P + ATP + NADH,GAPD,-130.5,NaN,NaN,337.113,467.5986,NaN,-146.628334
9,emp,cytosol,G3P == PEP,ENO,-168.3,NaN,NaN,1110.7242,1279.2996,NaN,-146.628334


### Acetate GSM FBA

In [10]:
# minimal medium with acetate as sole carbon source
model.medium = {
  'EX_ac(e)': 100.0, # 100 mmol of acetate
  'EX_h2o(e)': 10000.0,
  'EX_h(e)': 10000.0,
  'EX_nh4(e)': 10000.0,
  'EX_o2(e)': 10000.0,
  'EX_pi(e)': 10000.0,
  'EX_so4(e)': 10000.0,
}

with model:
  # update the model to use the specified medium
  medium = model.medium
  [print(model.medium[m], m) for m in model.medium]

  # find the optimal solution
  acetate_fba_solution = cobra.flux_analysis.pfba(model)

  # make a list of dictionaries with the reaction id, name, flux, and absolute flux
  reactions = []
  for reaction_id, flux in acetate_fba_solution.fluxes.items():
    reactions.append({
      'reaction_id': reaction_id,
      'reaction_name': model.reactions.get_by_id(reaction_id).name,
      'full_reaction': model.reactions.get_by_id(reaction_id).reaction,
      'flux': flux,
      'absolute_flux': abs(flux), # use for sorting, then drop
    })

  # make a dataframe from the list of dictionaries
  acetate_gsm_fba_df = pd.DataFrame(reactions)

  # sort the dataframe by absolute flux
  acetate_gsm_fba_df = acetate_gsm_fba_df.sort_values(by=['absolute_flux'], ascending=False)

  # drop the absolute flux column
  acetate_gsm_fba_df = acetate_gsm_fba_df.drop(columns=['absolute_flux'])

  # save the dataframe to a csv file
  acetate_gsm_fba_df.to_csv('../results/fba_gsm_fluxes/acetate_gsm_fba.csv', index=False)

  display(acetate_gsm_fba_df.head(30))


100.0 EX_ac(e)
10000.0 EX_h2o(e)
10000.0 EX_h(e)
10000.0 EX_nh4(e)
10000.0 EX_o2(e)
10000.0 EX_pi(e)
10000.0 EX_so4(e)


,reaction_id,reaction_name,full_reaction,flux
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,288.370347
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-254.968998
617,CYOR_u6m,ubiquinol 6 cytochrome c reductase,2.0 ficytc[m] + 1.5 h[m] + q6h2[m] --> 2.0 foc...,218.157527
959,H2Ot,H2O transport via diffusion,h2o[e] <=> h2o[c],-145.294979
421,EX_h2o(e),H2O exchange,h2o[e] <=>,145.294979
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],132.557867
846,NADH2_u6m,NADH dehydrogenase mitochondrial,h[m] + nadh[m] + q6[m] --> nad[m] + q6h2[m],129.024067
391,EX_co2(e),CO2 exchange,co2[e] -->,113.961697
957,CO2t,CO2 transporter via diffusion,co2[e] <=> co2[c],-113.961697
444,EX_o2(e),O2 exchange,o2[e] <=>,-110.374569


### Acetate Central Metabolism FBA

In [11]:
# load acetate 13C MFA data
acetate_13c_mfa_df = pd.read_csv('../data/13c_mfa/acetate_13c_mfa.csv')

# only keep the reactions that are mapped to the GSM
acetate_13c_mfa_df.dropna(subset = ["reaction_ids"], inplace=True)

# add the GSM flux predictions to the 13C-MFA dataframe
acetate_central_fba_df = add_column_to_13c_flux_df(acetate_13c_mfa_df, acetate_gsm_fba_df, 'pFBA Flux')

# save the central fluxes as .csv
acetate_central_fba_df.to_csv('../results/fba_central_fluxes/acetate_central_fba.csv', index=False)

acetate_central_fba_df

,pathway,compartment,13C_mfa_reaction,reaction_ids,13C_mfa_flux,13C_mfa_flux_95%_lower_bound,13C_mfa_flux_95%_upper_bound,Forward Flux,Backward Flux,Stdev,pFBA Flux
0,substrate_uptake,cytosol,Glucose(substrate) + ATP == G6P,GLCt1,0.0,NaN,NaN,[],[],NaN,0.000000
1,substrate_uptake,cytosol,Glycerol(substrate) == GLYC,reverse_GLYCt,0.0,NaN,NaN,[],[],NaN,0.000000
2,substrate_uptake,cytosol,Acetate(substrate) == AC,ACt2r,100.0,NaN,NaN,[],[],NaN,100.000000
3,substrate_uptake,cytosol,oleic_acid(substrate) == AC,OCDCEAt,0.0,NaN,NaN,[],[],NaN,0.000000
4,emp,cytosol,G6P == F6P,PGI,-8.6,NaN,NaN,7.7935,16.3725,NaN,-2.356175
5,emp,cytosol,F6P + ATP == FBP,PFK or reverse_FBP,-7.3,NaN,NaN,8.67E+03,8.68E+03,NaN,-5.328969
6,emp,cytosol,FBP == DHAP + GAP,FBA,-7.3,NaN,NaN,28.0246,35.3659,NaN,-5.328969
7,emp,cytosol,DHAP == GAP,TPI,-7.3,NaN,NaN,0,7.3413,NaN,-5.552589
8,emp,cytosol,GAP == G3P + ATP + NADH,GAPD,-14.5,NaN,NaN,37.457,51.9554,NaN,-11.253005
9,emp,cytosol,G3P == PEP,ENO,-18.7,NaN,NaN,123.4138,142.1444,NaN,-11.253005
